# Utilidades para limpiar o consolidar datos 
**- Ayudante: Thomas Buddemberg Astudillo**

**- Mail: tbuddemberg@alumnos.uai.cl**

**+56 9 7532 8800**

Veremos algunas funcionalidades de pandas para limpiar y consolidar datos.

In [1]:
import pandas as pd
import numpy as np # Lo usaremos para generar NaNs

## Valores fuera del tipo esperado
Es muy usual encontrar, en fuentes de datos, valores que no corresponden al tipo esperado. Por ejemplo, cuando tenemos una columna de números, donde algunos de los elementos son strings.

In [2]:
edad = pd.DataFrame({"Peso (Kg)": [75,"     64",87, 59]},
    index = ["Francisco", "Marta", "Pablo", "Camila"])
edad

,Peso (Kg)
Francisco,75
Marta,64
Pablo,87
Camila,59


In [3]:
edad["Peso (Kg)"] + 100

TypeError: can only concatenate str (not "int") to str

In [4]:
edad.dtypes

Peso (Kg)    object
dtype: object

En estos casos, el atributo `.astype()` nos permite reinterpretar (usualmente transformando) un tipo de dato en otro.

In [5]:
edad["Peso (Kg)"] = edad["Peso (Kg)"].astype(int)
edad

,Peso (Kg)
Francisco,75
Marta,64
Pablo,87
Camila,59


In [6]:
edad["Peso (Kg)"] + 100

Francisco    175
Marta        164
Pablo        187
Camila       159
Name: Peso (Kg), dtype: int32

Pero esto va a funcionar mientras el valor sea transformable. Muchas veces nos encontraremos con casos no trivialmente transformables:

In [7]:
edades = pd.DataFrame({"Peso (Kg)":[75,"LoL",87, 59]},
    index = ["Francisco", "Marta", "Pablo", "Camila"])

In [8]:
edades

,Peso (Kg)
Francisco,75
Marta,LoL
Pablo,87
Camila,59


In [9]:
edades["Peso (Kg)"] = edades["Peso (Kg)"].astype(int)

ValueError: invalid literal for int() with base 10: 'LoL'

En este caso podemos usar `pd.to_numeric()`, el cual es muy flexible y permite decidir qué hacer en caso de no poder transformar un dato. En particular podemos usar `errors="coerce"`, lo que insertará un `NaN` cuando no pueda convertir al tipo indicado.

In [10]:
pd.to_numeric(edades["Peso (Kg)"], errors="coerce") # Coerce = Obligar

Francisco    75.0
Marta         NaN
Pablo        87.0
Camila       59.0
Name: Peso (Kg), dtype: float64

Con lo que sabemos podemos convertir esos `NaN` en ceros y convertir esa columna en enteros (actualmente esta en float):

In [11]:
pd.to_numeric(edades["Peso (Kg)"], errors="coerce").fillna(0)

Francisco    75.0
Marta         0.0
Pablo        87.0
Camila       59.0
Name: Peso (Kg), dtype: float64

In [12]:
pd.to_numeric(edades["Peso (Kg)"], errors="coerce").fillna(0).astype("int")

Francisco    75
Marta         0
Pablo        87
Camila       59
Name: Peso (Kg), dtype: int32

# Limpiando strings
Además de números incorrectos, muchas veces tendremos textos (strings, categorías o valores nominales), con distintas clases de ruido.

La gran mayoría los podremos limpiar con las funciones de string incluidas en `DataFrame.str`, que son equivalentes a las funciones puras de strings en python. 

Por ejemplo:

In [13]:
Frases = pd.Series([
    "No importan los años de vida, sino la vida de los años",
    "       De las dificultades nacen milagros",
    "###A#l######g###u######nos #bu####s#ca######n u######n ######m#un#######do ######m#ás b##o####nito, ot##ros lo cr###e#an##",
    "Un buen viajante no tiene planes",
    "S######é #####tu m####i###########s##mo las ###co##pias### se ve##nden bar##atas"])

In [14]:
Frases

0    No importan los años de vida, sino la vida de ...
1                   De las dificultades nacen milagros
2    ###A#l######g###u######nos #bu####s#ca######n ...
3                     Un buen viajante no tiene planes
4    S######é #####tu m####i###########s##mo las ##...
dtype: object

In [15]:
Frases.str.replace("#","")

0    No importan los años de vida, sino la vida de ...
1                   De las dificultades nacen milagros
2    Algunos buscan un mundo más bonito, otros lo c...
3                     Un buen viajante no tiene planes
4             Sé tu mismo las copias se venden baratas
dtype: object

Para limpiezas más sofisticadas, siempre podemos definir una función en python y aplicarla a toda la columna:

In [16]:
def clean(frases):
    frases = frases.replace("#","")
    frases = "¡"+frases+"!"
    return frases

Frases.apply(clean)

0    ¡No importan los años de vida, sino la vida de...
1          ¡       De las dificultades nacen milagros!
2    ¡Algunos buscan un mundo más bonito, otros lo ...
3                   ¡Un buen viajante no tiene planes!
4           ¡Sé tu mismo las copias se venden baratas!
dtype: object

¡Nos faltó eliminar los espacios en blanco! Aquí usaremos `str.strip()`:

In [17]:
Frases.str.strip().apply(clean)

0    ¡No importan los años de vida, sino la vida de...
1                 ¡De las dificultades nacen milagros!
2    ¡Algunos buscan un mundo más bonito, otros lo ...
3                   ¡Un buen viajante no tiene planes!
4           ¡Sé tu mismo las copias se venden baratas!
dtype: object

In [92]:
profesores = pd.read_csv('Profesores_UAI.csv')
profesores

,Unnamed: 0,Profesor,Título,Sala,Sede
0,0,Rod0lfo Abanto,Magíster en Estadística,323-D,Peñalolén
1,1,S@amuel Varas,PhD en Tecnologías de la Información,NaN,Peñalolén
2,0,Francisco Duque,Master of Energy Systems,103-D,Peñalolén
3,1,Rodolfo Abanto,Magíster en Estadística,323-D,Peñalolén
4,2,Eduardo Moreno,PhD Sciences of Engineering,304a-D,Peñalolén
5,3,Luis Aburto,Doctor en Sistemas de Ingeniería,311-D,Peñalolén
6,4,Viviana Barile,Doctora en Matemáticas,315-D,Peñalolén
7,5,Sebastián Moreno,PhD in Computer Science,212-A,Viña del Mar
8,6,Hugo Caerols,Doctor en Ciencias exactas con Mención en Mate...,310-D,Peñalolén
9,7,Rafael Cereceda,Magíster en Ingeniería de Negocios con mención...,322-D,Peñalolén


In [93]:
profesores.drop(columns='Unnamed: 0', inplace=True)
profesores.columns

Index(['Profesor', 'Título', 'Sala', 'Sede'], dtype='object')

# Arreglar datos

In [87]:
profesores.head(2)

,Profesor,Título,Sala,Sede
0,Rod0lfo Abanto,Magíster en Estadística,323-D,Peñalolén
1,S@amuel Varas,PhD en Tecnologías de la Información,NaN,Peñalolén


Esto es fácil de detectar ya que son **pocos datos**, pero si tuviesemos muchos datos?

In [88]:
lista = ['1', '2','3','4','5','6','7','8','9','0','@','/','_','-','+','='] # etc 

In [94]:
for j in profesores['Profesor']:
    for i in lista:
        if j.find(i) == -1:
            pass
        else:
            eliminar = profesores['Profesor'][profesores['Profesor'] == j].index
            print(f'\n{eliminar}')
            print(f'{profesores[profesores["Profesor"] == j]}')
            profesores = profesores.drop(eliminar)


Int64Index([0], dtype='int64')
         Profesor                   Título   Sala       Sede
0  Rod0lfo Abanto  Magíster en Estadística  323-D  Peñalolén

Int64Index([1], dtype='int64')
        Profesor                                Título Sala       Sede
1  S@amuel Varas  PhD en Tecnologías de la Información  NaN  Peñalolén


In [74]:
profesores

,Profesor,Título,Sala,Sede
2,Francisco Duque,Master of Energy Systems,103-D,Peñalolén
3,Rodolfo Abanto,Magíster en Estadística,323-D,Peñalolén
4,Eduardo Moreno,PhD Sciences of Engineering,304a-D,Peñalolén
5,Luis Aburto,Doctor en Sistemas de Ingeniería,311-D,Peñalolén
6,Viviana Barile,Doctora en Matemáticas,315-D,Peñalolén
7,Sebastián Moreno,PhD in Computer Science,212-A,Viña del Mar
8,Hugo Caerols,Doctor en Ciencias exactas con Mención en Mate...,310-D,Peñalolén
9,Rafael Cereceda,Magíster en Ingeniería de Negocios con mención...,322-D,Peñalolén
10,Moreno Bevilacqua,PhD en Estadística,NaN,Viña del Mar
11,Florencia Darrigrandi,Magíster en Estadística,210-D,Peñalolén


# Duplicados

In [95]:
profesores.duplicated().sum()

1

In [96]:
profesores[profesores.duplicated() == True]

,Profesor,Título,Sala,Sede
19,Rodolfo Abanto,Magíster en Estadística,323-D,Peñalolén


In [97]:
profesores[profesores['Profesor'] == 'Rodolfo Abanto']

,Profesor,Título,Sala,Sede
3,Rodolfo Abanto,Magíster en Estadística,323-D,Peñalolén
16,Rodolfo Abanto,Master in Statistics,323-D,Peñalolén
18,Rodolfo Abanto,Magíster en Estadística,310-E,Peñalolén
19,Rodolfo Abanto,Magíster en Estadística,323-D,Peñalolén


## Duplicados por Columna

In [98]:
profesores.columns

Index(['Profesor', 'Título', 'Sala', 'Sede'], dtype='object')

In [99]:
profesores[profesores['Título'].duplicated() == True]

,Profesor,Título,Sala,Sede
11,Florencia Darrigrandi,Magíster en Estadística,210-D,Peñalolén
18,Rodolfo Abanto,Magíster en Estadística,310-E,Peñalolén
19,Rodolfo Abanto,Magíster en Estadística,323-D,Peñalolén


No muy bueno ya que puede haber profesores con el mismo `Titulo`.

Podemos probar con la `Sala`, ya que no deberian haber dos profesores en una misma sala.

In [100]:
profesores[profesores['Sala'].duplicated() == True]

,Profesor,Título,Sala,Sede
12,Felipe Lagos,Ph.D en Investigación de Operaciones,NaN,Peñalolén
13,Samuel Varas,PhD en Tecnologías de la Información,NaN,Peñalolén
15,Javier Lopatin,Doctor en Recursos Naturales,NaN,Peñalolén
16,Rodolfo Abanto,Master in Statistics,323-D,Peñalolén
17,Viviana Barile,PhD in Mathematics,315-D,Peñalolén
19,Rodolfo Abanto,Magíster en Estadística,323-D,Peñalolén


Existen profesores sin salas, por lo que no es un buen filtro.

In [101]:
profesores[profesores['Profesor'].duplicated() == True]

,Profesor,Título,Sala,Sede
16,Rodolfo Abanto,Master in Statistics,323-D,Peñalolén
17,Viviana Barile,PhD in Mathematics,315-D,Peñalolén
18,Rodolfo Abanto,Magíster en Estadística,310-E,Peñalolén
19,Rodolfo Abanto,Magíster en Estadística,323-D,Peñalolén


**¡Este si es un buen filtro!**

Ya que logramos identificar los valores duplicados

In [102]:
profesores['Profesor'].drop_duplicates()

2           Francisco Duque
3            Rodolfo Abanto
4            Eduardo Moreno
5               Luis Aburto
6            Viviana Barile
7          Sebastián Moreno
8              Hugo Caerols
9           Rafael Cereceda
10        Moreno Bevilacqua
11    Florencia Darrigrandi
12             Felipe Lagos
13             Samuel Varas
14            Thomas Ledger
15           Javier Lopatin
Name: Profesor, dtype: object

# Valores nulos

In [25]:
profesores.isna().sum()

Profesor    0
Título      0
Sala        4
Sede        0
dtype: int64

In [26]:
profesores[profesores.Sala.isna() == True]

,Profesor,Título,Sala,Sede
8,Moreno Bevilacqua,PhD en Estadística,NaN,Viña del Mar
10,Felipe Lagos,Ph.D en Investigación de Operaciones,NaN,Peñalolén
11,Samuel Varas,PhD en Tecnologías de la Información,NaN,Peñalolén
13,Javier Lopatin,Doctor en Recursos Naturales,NaN,Peñalolén


In [27]:
profesores.Sala.fillna('No posee Sala', inplace=True)

In [ ]:
profesores

,Profesor,Título,Sala,Sede
0,Francisco Duque,Master of Energy Systems,103-D,Peñalolén
1,Rodolfo Abanto,Magíster en Estadística,323-D,Peñalolén
2,Eduardo Moreno,PhD Sciences of Engineering,304a-D,Peñalolén
3,Luis Aburto,Doctor en Sistemas de Ingeniería,311-D,Peñalolén
4,Viviana Barile,Doctora en Matemáticas,315-D,Peñalolén
5,Sebastián Moreno,PhD in Computer Science,212-A,Viña del Mar
6,Hugo Caerols,Doctor en Ciencias exactas con Mención en Mate...,310-D,Peñalolén
7,Rafael Cereceda,Magíster en Ingeniería de Negocios con mención...,322-D,Peñalolén
8,Moreno Bevilacqua,PhD en Estadística,No posee Sala,Viña del Mar
9,Florencia Darrigrandi,Magíster en Estadística,210-D,Peñalolén


Otra tarea muy útil es romper strings para crear nuevas columnas:

In [ ]:
Nombres = profesores["Profesor"].str.split(expand = True)
Nombres.head()

,0,1
0,Francisco,Duque
1,Rodolfo,Abanto
2,Eduardo,Moreno
3,Luis,Aburto
4,Viviana,Barile


In [ ]:
print(Nombres[Nombres[0].duplicated()])
print(Nombres[Nombres[1].duplicated()])

Empty DataFrame
Columns: [0, 1]
Index: []
           0       1
5  Sebastián  Moreno


In [ ]:
profesores[profesores['Profesor'] == 'Varas']

,Profesor,Título,Sala,Sede


In [ ]:
profesores[Nombres[1] == 'Varas']

,Profesor,Título,Sala,Sede
11,Samuel Varas,PhD en Tecnologías de la Información,No posee Sala,Peñalolén


In [ ]:
profesores[Nombres[1] == 'Moreno']

,Profesor,Título,Sala,Sede
2,Eduardo Moreno,PhD Sciences of Engineering,304a-D,Peñalolén
5,Sebastián Moreno,PhD in Computer Science,212-A,Viña del Mar


In [ ]:
Salas = profesores["Sala"].str.split("-", expand = True)
Salas

,0,1
0,103,D
1,323,D
2,304a,D
3,311,D
4,315,D
5,212,A
6,310,D
7,322,D
8,No posee Sala,None
9,210,D


In [ ]:
profesores[Salas[1] == 'D']

,Profesor,Título,Sala,Sede
0,Francisco Duque,Master of Energy Systems,103-D,Peñalolén
1,Rodolfo Abanto,Magíster en Estadística,323-D,Peñalolén
2,Eduardo Moreno,PhD Sciences of Engineering,304a-D,Peñalolén
3,Luis Aburto,Doctor en Sistemas de Ingeniería,311-D,Peñalolén
4,Viviana Barile,Doctora en Matemáticas,315-D,Peñalolén
6,Hugo Caerols,Doctor en Ciencias exactas con Mención en Mate...,310-D,Peñalolén
7,Rafael Cereceda,Magíster en Ingeniería de Negocios con mención...,322-D,Peñalolén
9,Florencia Darrigrandi,Magíster en Estadística,210-D,Peñalolén


In [ ]:
Salas[Salas[1].isnull() == True]

,0,1
8,No posee Sala,None
10,No posee Sala,None
11,No posee Sala,None
13,No posee Sala,None


In [ ]:
profesores[Salas[1].isnull() == True]

,Profesor,Título,Sala,Sede
8,Moreno Bevilacqua,PhD en Estadística,No posee Sala,Viña del Mar
10,Felipe Lagos,Ph.D en Investigación de Operaciones,No posee Sala,Peñalolén
11,Samuel Varas,PhD en Tecnologías de la Información,No posee Sala,Peñalolén
13,Javier Lopatin,Doctor en Recursos Naturales,No posee Sala,Peñalolén


In [ ]:
profesores["Profesor"] + " / " + profesores["Sala"] + ' / ' + profesores['Sede']

0                  Francisco Duque / 103-D / Peñalolén
1                   Rodolfo Abanto / 323-D / Peñalolén
2                  Eduardo Moreno / 304a-D / Peñalolén
3                      Luis Aburto / 311-D / Peñalolén
4                   Viviana Barile / 315-D / Peñalolén
5              Sebastián Moreno / 212-A / Viña del Mar
6                     Hugo Caerols / 310-D / Peñalolén
7                  Rafael Cereceda / 322-D / Peñalolén
8     Moreno Bevilacqua / No posee Sala / Viña del Mar
9            Florencia Darrigrandi / 210-D / Peñalolén
10            Felipe Lagos / No posee Sala / Peñalolén
11            Samuel Varas / No posee Sala / Peñalolén
12                   Thomas Ledger / 302-E / Peñalolén
13          Javier Lopatin / No posee Sala / Peñalolén
dtype: object

## Eliminando duplicados
En la práctica tendremos datos repetidos, que usualmente querremos eliminar bajo algún criterio (por ejemplo, quedarnos con el primero o el último de los datos de algún indetificador).

In [ ]:
datos = pd.DataFrame({
    "x": [5, 11, 24, 11, 28, 11, 8],
    "y": [11, 2, 7, 2, 10, 2,  2]
})
datos

,x,y
0,5,11
1,11,2
2,24,7
3,11,2
4,28,10
5,11,2
6,8,2


In [ ]:
datos.duplicated().sum()

2

In [ ]:
datos.drop_duplicates()

,x,y
0,5,11
1,11,2
2,24,7
4,28,10
6,8,2


In [ ]:
data = pd.DataFrame({
    "x": [6, 1, 21, 5, 9],
    "y": [3, 5, 3, 44, 6]
})
data

,x,y
0,6,3
1,1,5
2,21,3
3,5,44
4,9,6


`drop_duplicates` puede recibir una lista de las columnas donde buscar duplicados para decidir si botar o no la fila.

In [ ]:
data.drop_duplicates(["y"])

,x,y
0,6,3
1,1,5
3,5,44
4,9,6


`keep` es el parámetro que nos permite indicar qué dato queremos que sobreviva (puede ser `'first'`, `'last'` o `False`).

In [ ]:
data.drop_duplicates(["y"], keep='last')

,x,y
1,1,5
2,21,3
3,5,44
4,9,6


### Reemplazando valores
Ya hemos visto `map()`, el método `replace` cumple una función similar. Es menos flexible, pero sintácticamente más sencillo:

In [ ]:
monto = pd.DataFrame(
    {"monto": [2, -999, 3, -999, -1000, 5]}
)
monto

,monto
0,2
1,-999
2,3
3,-999
4,-1000
5,5


In [ ]:
monto.replace(-999, np.nan)

,monto
0,2.0
1,NaN
2,3.0
3,NaN
4,-1000.0
5,5.0


In [ ]:
monto.replace([-999,-1000], -1)

,monto
0,2
1,-1
2,3
3,-1
4,-1
5,5


In [ ]:
monto.replace([-999,-1000], [9, 10])

,monto
0,2
1,9
2,3
3,9
4,10
5,5


## Juntar data frames
Otra situación muy común es recibir datos contenidos en distintos dataframes.
### Primer caso: concatenar observaciones

In [ ]:
seccion11 = pd.DataFrame({'Nombre': ['Mauricio', 'Diana'], 
                         'Email': ['mau@uai.cl', 'di@uai.cl']})

seccion2 = pd.DataFrame({'Nombre': ['Carla', 'Alexis', 'Marcela'], 
                         'Email': ['c@uai.cl', 'al@uai.cl', 'mar@uai.cl']})

In [ ]:
seccion1

,Nombre,Email
0,Mauricio,mau@uai.cl
1,Diana,di@uai.cl


In [ ]:
seccion2

,Nombre,Email
0,Carla,c@uai.cl
1,Alexis,al@uai.cl
2,Marcela,mar@uai.cl


In [ ]:
pd.concat([seccion1, seccion2], ignore_index = False) # usar ignore_index = True para resetear index

,Nombre,Email
0,Mauricio,mau@uai.cl
1,Diana,di@uai.cl
0,Carla,c@uai.cl
1,Alexis,al@uai.cl
2,Marcela,mar@uai.cl


In [ ]:
pd.concat([seccion1, seccion2], ignore_index = True)

,Nombre,Email
0,Mauricio,mau@uai.cl
1,Diana,di@uai.cl
2,Carla,c@uai.cl
3,Alexis,al@uai.cl
4,Marcela,mar@uai.cl


¿Qué pasa si uno de los dataframes tiene una variable adicional?

In [ ]:
seccion1_1 = pd.DataFrame({'Nombre': ['Mauricio', 'Diana'], 
                           'Email': ['mau@uai.cl', 'di@uai.cl'],
                        'edad': [21, 20]})
seccion1_1

,Nombre,Email,edad
0,Mauricio,mau@uai.cl,21
1,Diana,di@uai.cl,20


In [ ]:
pd.concat([seccion1_1, seccion2])

,Nombre,Email,edad
0,Mauricio,mau@uai.cl,21.0
1,Diana,di@uai.cl,20.0
0,Carla,c@uai.cl,NaN
1,Alexis,al@uai.cl,NaN
2,Marcela,mar@uai.cl,NaN


### Caso 2: juntar data frames según una o más columnas en común

In [ ]:
alumnos = pd.concat([seccion1, seccion2])
alumnos

NameError: name 'seccion1' is not defined

In [ ]:
grupos = pd.DataFrame({'Email': ['mau@uai.cl', 'di@uai.cl', 'c@uai.cl', 'al@uai.cl', 
                                 'mar@uai.cl', 'carlos@uai.cl', 'alejandro@uai.cl'],
                     'Grupo': [2, 3, 1, 5, 
                               3, 3, 2]})
grupos

,Email,Grupo
0,mau@uai.cl,2
1,di@uai.cl,3
2,c@uai.cl,1
3,al@uai.cl,5
4,mar@uai.cl,3
5,carlos@uai.cl,3
6,alejandro@uai.cl,2


In [ ]:
pd.merge(alumnos, grupos, on = 'Email', how = 'inner')

,Nombre,Email,Grupo
0,Mauricio,mau@uai.cl,2
1,Diana,di@uai.cl,3
2,Carla,c@uai.cl,1
3,Alexis,al@uai.cl,5
4,Marcela,mar@uai.cl,3


# Fin